<a href="https://colab.research.google.com/github/HaroldoFV/hungarian_method/blob/main/hungarian_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Trabalhadores
workers = ["Ana", "Bruno", "Carlos"]

# Tarefas
tasks = ["Tarefa 1", "Tarefa 2", "Tarefa 3"]

# Matriz de custos
costs = [
    [15, 31, 12],
    [45, 28, 29],
    [22, 12, 45]
]

In [2]:
def reduce_rows(costs):
    for row in costs:
        min_val = min(row)
        for j in range(len(row)):
            row[j] -= min_val
    return costs

costs_reduced = reduce_rows(costs)
costs_reduced

[[3, 19, 0], [17, 0, 1], [10, 0, 33]]

In [3]:
def reduce_cols(costs):
    for j in range(len(costs[0])):
        min_val = min(row[j] for row in costs)
        for i in range(len(costs)):
            costs[i][j] -= min_val
    return costs

costs_reduced_cols = reduce_cols(costs_reduced)
costs_reduced_cols

[[0, 19, 0], [14, 0, 1], [7, 0, 33]]

In [4]:
def mark_zeros(matrix):
    rows = len(matrix)
    cols = len(matrix[0])

    covered_rows = set()
    covered_cols = set()

    while True:
        # Count uncovered zeros in each row and column
        row_zeros = []
        for i in range(rows):
          count = 0
          if i not in covered_rows:
            count += sum(1 for j in range(cols) if matrix[i][j] == 0 and j not in covered_cols)
          row_zeros.append(count)

        # Count uncovered zeros in each column
        col_zeros = []
        for j in range(cols):
          count = 0
          if j not in covered_cols:
            count += sum(1 for i in range(rows) if matrix[i][j] == 0 and i not in covered_rows)
          col_zeros.append(count)

        # If no more uncovered zeros, break
        if max(row_zeros) == 0 and max(col_zeros) == 0:
            break

        # Find the row or column with the maximum number of uncovered zeros
        max_row_zeros = max((count, i) for i, count in enumerate(row_zeros))
        max_col_zeros = max((count, j) for j, count in enumerate(col_zeros))

        if max_row_zeros[0] >= max_col_zeros[0] and max_row_zeros[1] not in covered_rows:
            covered_rows.add(max_row_zeros[1])
        elif max_col_zeros[1] not in covered_cols:
            covered_cols.add(max_col_zeros[1])


    return list(covered_rows), list(covered_cols)

# Testing with the provided matrix
marked = mark_zeros(costs_reduced_cols)
marked


([0], [1])

In [5]:
def adjust_matrix(cost_matrix, covered):
    covered_rows, covered_cols = covered
    min_val = float('inf')

    rows, cols = len(cost_matrix), len(cost_matrix[0])

    # Step 1: Find the smallest value from the uncovered cells
    for i in range(rows):
        for j in range(cols):
            if i not in covered_rows and j not in covered_cols:
                min_val = min(min_val, cost_matrix[i][j])

    # Step 2: Subtract this value from cells of the unmarked rows
    for i in range(rows):
        if i not in covered_rows:
            for j in range(cols):
                cost_matrix[i][j] -= min_val

    # Step 3: Add this value to cells of the marked columns
    for j in covered_cols:
        for i in range(rows):
            cost_matrix[i][j] += min_val

    return cost_matrix

adjusted = adjust_matrix(costs_reduced_cols, marked)
adjusted

[[0, 20, 0], [13, 0, 0], [6, 0, 32]]

In [6]:
def find_assignments(matrix):
    n = len(matrix)
    assignments = [-1] * n  # Initialize assignments

    def is_safe(row, col):
        for i in range(row):
            if assignments[i] == col or matrix[row][col] != 0:
                return False
        return True

    def solve(row):
        if row == n:
            return True
        for col in range(n):
            if is_safe(row, col):
                assignments[row] = col
                if solve(row + 1):
                    return True
                assignments[row] = -1
        return False

    if solve(0):
        return [(i, assignments[i]) for i in range(n)]
    else:
        return []

assignments = find_assignments(adjusted)
assignments

[(0, 0), (1, 2), (2, 1)]

In [7]:
def hungarian_method(workers, tasks, costs):
    costs_reduced = reduce_rows(costs)
    costs_reduced_cols = reduce_cols(costs_reduced)

    while True:
        marked = mark_zeros(costs_reduced_cols)
        if len(marked[0]) + len(marked[1]) >= len(costs):
            break
        costs_reduced_cols = adjust_matrix(costs_reduced_cols, marked)

    assignments = find_assignments(costs_reduced_cols)

    # Return the assignments in terms of workers and tasks
    return [(workers[i], tasks[j]) for i, j in assignments]

# Testing the hungarian_method with the provided data
hungarian_method(workers, tasks, costs)

[('Ana', 'Tarefa 1'), ('Bruno', 'Tarefa 3'), ('Carlos', 'Tarefa 2')]